In [4]:
from dotenv import load_dotenv

load_dotenv(".env.local")

True

In [7]:
import os

pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [98]:
from langchain.chains import RetrievalQA  
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=os.environ.get('OPENAI_API_KEY'),
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [99]:
query1 = """What experiences do you have with databases?"""

query2 = """Describe some projects you've worked on that involve machine learning?"""\

print("Query 1\n")
print("Chat with knowledge:")
print(qa.invoke(query1).get("result"))

print("\nQuery 2\n")
print("Chat with knowledge:")
print(qa.invoke(query2).get("result"))


Query 1

Chat with knowledge:
I am an AI assistant and do not have personal experiences.

Query 2

Chat with knowledge:
I don't have the ability to work on projects or have personal experiences.


## Embedding nodes into the PineconeDB

In [25]:
from dotenv import load_dotenv

load_dotenv(".env.local")

True

In [29]:
from neo4j import GraphDatabase
import os

uri = os.getenv("NEO4J_URL")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
driver = GraphDatabase.driver(uri, auth=(username, password))
driver

In [30]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pinecone_api_key = os.getenv("PINECONE_API_KEY")

index_name = "avd-bot"
pc = Pinecone(api_key=pinecone_api_key)

model_name = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(
  model=model_name,
)
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)


In [31]:
node_labels = ["Experience", "Descriptions", "Skills"]

def get_nodes(driver, label):
    with driver.session() as session:
        result = session.run(
            f"MATCH (n:{label}) RETURN n"
        )
        return [record["n"] for record in result]
    
for label in node_labels:
    nodes = get_nodes(driver, label)
    metadata = []
    texts = []

    for node in nodes:
        metadata.append({
            "element_id": node.element_id,
            "label": label, 
            "property": "label",
            "value": label
        })
        texts.append(label)
        for key in node.keys():
            if key == "id":
                continue
            metadata.append({
                "element_id": node.element_id,
                "label": label, 
                "property": key,
                "value": node[key]
            })
            texts.append(node[key])
    
    vector_store.add_texts(texts, metadatas=metadata)




In [22]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=os.environ.get('OPENAI_API_KEY'),
    model_name='gpt-4o',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [23]:
results = vector_store.similarity_search_with_score("Do you have experience with databases?")
max_score = max([result[1] for result in results])
top_nodes = [result[0] for result in results if result[1] == max_score]
top_nodes

[Document(id='5d21843d-37cd-49b2-9544-7eb29d9855fd', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:280', 'label': 'Skills', 'property': 'type', 'value': 'Database'}, page_content='Database'),
 Document(id='f395bac5-7b75-4aa0-8d29-ebd27001ee4a', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:278', 'label': 'Skills', 'property': 'type', 'value': 'Database'}, page_content='Database'),
 Document(id='ade0c48e-4512-4107-885f-7af025247de6', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:279', 'label': 'Skills', 'property': 'type', 'value': 'Database'}, page_content='Database')]

In [24]:
vector_store.similarity_search_with_score("What did you learn in your masters?")

[(Document(id='491d084c-d7c5-4bd6-b4f9-043631e796bc', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:245', 'label': 'Experience', 'property': 'subtitle', 'value': "Master's in Data Science"}, page_content="Master's in Data Science"),
  0.538854599),
 (Document(id='1a9a614f-4a00-4d2e-9afe-84d4eacd748a', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:248', 'label': 'Experience', 'property': 'subtitle', 'value': 'Bacheclor of Science in Computer Engineering'}, page_content='Bacheclor of Science in Computer Engineering'),
  0.295194149),
 (Document(id='079bb22a-f47a-4726-9c94-f576c9e07e69', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:265', 'label': 'Descriptions', 'property': 'description', 'value': 'Mentored other engineers through code reviews and pairing sessions.,'}, page_content='Mentored other engineers through code reviews and pairing sessions.,'),
  0.269667864),
 (Document(id='97316541-9703-482e-930b-6b2faede692d', metadata={'e

In [138]:
qa.invoke("What did you learn in your masters?").get("result")

"I don't have personal experiences or the ability to pursue a master's degree. However, I can provide information on what a typical master's program might cover in various fields if that would be helpful."